Team id: 3<br>
Div - C<br>
Semester - 5th<br>
School Of Computer Science and Engineering

**5DMACP06 Richter's Predictor Modelling Earthquake**

Problem Statement :-
Based on aspects of building location and construction, the goal is to predict the level of damage to buildings
caused by the 2015 Gorkha earthquake in Nepal.

Team Leader:<br>
Nitin Verma     Roll No: 148, USN: 01FE17BCS125

Team Members:<br>
Puneet Gupta    Roll No: 166, USN: 01FE17BCS144<br>
Rahetul Asquin  Roll No: 170, USN: 01FE17BCS148<br>
Ritbik Bharti   Roll No: 179, USN: 01FE17BCS158<br>

In [ ]:
#Importing necessary libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestClassifier

/usr/lib/python3/dist-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [ ]:
#Loading the datasets

train_values = pd.read_csv('Data/train_values.csv')
train_labels = pd.read_csv('Data/train_labels.csv')
test_values = pd.read_csv('Data/test_values.csv')
submission = pd.read_csv('Data/submission_format.csv')
train_values.head()

In [ ]:
#Display the first 5 tuples of training labels

train_labels.head()

In [ ]:
#Display the first 5 tuples of testing values

test_values.head()

In [ ]:
#Display the first 5 tuples of submission format

submission.head()

** Getting a concise summary of the training data **

In [ ]:
train_values.info()

**Analysis:-**
- 31 numerical attributes
- 8 categorical attributes

** Checking for any missing values **

In [ ]:
#Columns With Missing Values as a List

def getColumnsWithMissingValuesList(df):
    return [col for col in df.columns if df[col].isnull().any()] 

getColumnsWithMissingValuesList(train_values)

**Conclusion:**
- There is no any missing data in the training data

** Checking for percentage of missing values of each attribute**

In [ ]:
#Get Missing Values Info

def missingValuesInfo(df):
    total = df.isnull().sum().sort_values(ascending = False)
    percent = round(df.isnull().sum().sort_values(ascending = False)/len(df)*100, 2)
    temp = pd.concat([total, percent], axis = 1,keys= ['Total', 'Percentage'])
    return temp.loc[(temp['Total'] > 0)]

missingValuesInfo(train_values)

**Conclusion:**
- There is no any missing data in the training data
- Therefore nill percentages of missing values of each attribute

** Checking for any NaN values**

In [ ]:
train_values.isnull().sum().sum()

**Analysis:**
- There is no any NaN value.

In [ ]:
#Columns of object Data Type (Categorical Columns) as a List

def getObjectColumnsList(df):
    return [cname for cname in df.columns if df[cname].dtype == "object"]

cat_cols = getObjectColumnsList(train_values)
cat_cols

In [ ]:
#Columns that Contain Numeric Values as a List

def getNumericColumnsList(df):
    return [cname for cname in df.columns if df[cname].dtype in ['int64', 'float64']]

num_cols = getNumericColumnsList(train_values)
num_cols

In [ ]:
#Joining labels data in training data
train_values['damage_grade'] = pd.Series(train_labels['damage_grade'])

**Corelation between all attributes**

In [ ]:
df = train_values.corr(method ='pearson')
df

**Corelation between dependent and independent attribute**
- The Pearson correlation coefficient (named for Karl Pearson) can be used to summarize the strength of the linear relationship between two data samples. 

In [ ]:
df = train_values.corr(method ='pearson')
df['damage_grade']

**To view some basic statistical details like percentile, mean, std etc. of the training data.**

In [ ]:
train_values.describe()

**Analysis:-**
* area_percentage
    - Min is 1 and max is 100, thats a big difference in percentages
* age
    - Mean age is 26 wheres some values are near to 1000
    - Max values are less than 100
* height_percentage
    - Max is 32 and mean is 5, hence large deviation of max value

**Understanding the target variable**

In [ ]:
sns.countplot(x = 'damage_grade', data = train_labels)
plt.title('Number of Buildings with Each Damage Grade');

In [ ]:
print("1: ",sum(train_values.damage_grade==1))
print("2: ",sum(train_values.damage_grade==2))
print("3: ",sum(train_values.damage_grade==3))

**Analysis:-**
- Most houses have damage grade 2
- Damage grade 1 has less number of houses

**Analyzing numerical attributes**

In [ ]:
num_col = ['geo_level_1_id',
 'geo_level_2_id',
 'geo_level_3_id',
 'count_floors_pre_eq',
 'age',
 'area_percentage',
 'height_percentage']

In [ ]:
train_values[num_col].hist( figsize=(20,10), layout=(3, 3));

**Analysis:-**
- The age of houses is less than 100, except few being around 1000
- The height percentage is more in the range 2 to 8
- The area_percentage is more dense in the range 0-19 , very few are around 40
- The count_floors_per_sq is amlost in the range 1-4, very few are > 5
- Geo_level_2_id and Geo_level_3_id are symmetrically distributed

**Analyzing Categorical Variables**

In [ ]:
fig, ax = plt.subplots(2, 4, figsize=(20, 10))
for variable, subplot in zip(cat_cols, ax.flatten()):
    sns.countplot(train_values[variable], ax=subplot)
    for label in subplot.get_xticklabels():
        label.set_rotation(90)

**Analysis:-**
- We got to know which categorical attribute is more significant after one hot encoding.

**Checking skewness of Data **

In [ ]:
print(train_values.skew())

**Analysis:-**
- If the value is near to zero then less skewed
- Age is a skewed attribute
- Features with external facilities like police, school etc are highly skewed

** Analysing the age attribute **

In [ ]:
sns.distplot(train_values.age, color='red')
plt.xlabel('Age')
sns.despine()

**Analysis:-**
- Most of the age values is < 100
- Some values is near to 1000